# Overview

This notebook provides a basic example of the process of working in PyDough to answer an analytics question. It highlights how PyDough can be leveraged to guide the development and experimentation process, with an emphasis on solving partial sub-problems.

We believe that is this approach is more composable and scalable. Rather than focusing on building a query to answer a question at hand, building question components allows more proportional scaling relative to the complexity at hand. The PyDough team is optimistic that such an approach can make human more productive effective and LLMs more accurate at solving the problem at hand.

The majority of this notebook will be focused on building up a more complex question piece by piece. At some places we will also deviate from this question to emphasize the powerful PyDough features that enable this type of experimentation.

In [1]:
%load_ext pydough_jupyter_extensions

In [2]:
import pydough

In [3]:
# Setup demo metadata
pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH")
pydough.active_session.connect_database("sqlite", database="../tpch.db");

## DEMO SETUP

For this demo we will be working in an example crafted to match the schema for the TPC-H benchmark. The actual data for this benchmark is generated in SQLite using the standard TPC-H data generation tools. The underlying schema of this data matches this example image from [TPC Benchmark H Standard Specification](https://www.tpc.org/tpc_documents_current_versions/pdf/tpc-h_v2.17.1.pdf).

![TPC-H schema from the Specification Document as of December 12, 2024](../images/tpc_h_schema.png)

In addition, our metadata then defines how these "collections" are known to interact with each other. One important feature is that we provide alternative names to reflect our organization's semantics. All PyDough code can use these names to make generating analytics queries closer to the underlying business questions. The important renamings for this example will be:
 * LineItems -> lines
 * PartSupp -> supply_records

In other use cases we can also define unique names for particular relationships (e.g. the joining of nations and regions), but we will not demonstrate these in this example to instead emphasize PyDough's naming flexibility.

In [4]:
# TODO: Show examples after Kian's Metadata PR showing these two example collections in the metadata.

## Problem Statement

We are going to build to solve a problem where we seek to identify the 5 lines that represent the smallest percentage of an order's revenue. We know the high level business question that we want to resolve, but not necessarily the path to get there. Here we will use PyDough to construct this solution one component at a time, rather than trying to start from composing queries on underlying tables.

Our first defintion will be to build our price defintion as this we be key to any solution. Based on underlying data we can define this price as being the original price after applying the discount.

In [5]:
%%pydough

price_def = extended_price*(1-discount)

This might seem shocking. We have defined `price_def` out of nowhere using an `extended_price` and `discount`. What has actually happened here is that we have generated what is called a `Contextless Expression`. This fundamental building block is the key to PyDough's composability.

On its own this expression doesn't mean anything. In fact if we inspect this object in regular PyDough we will see that PyDough itself has a lot of questions.

In [6]:
price_def

(?.extended_price * (1 - ?.discount))

As you see, PyDough now knows that this expression is composed of `extendedprice` and `discount`, but it doesn't know **WHICH** `extended_price` and `discount`. To ultimately develop a legal PyDough statement, we will need to bind uses of this expression to a context that can access `extended_price` and `discount`.

This might seem very minor, but this allows us to define definitions up front, allowing reuse in vastly different contexts.

Now our schema is flexible, but this price defintion is actually tied to the lines collection. Let's now combine incorporate lines.

In [7]:
%%pydough

line_price = lines(line_price=price_def).line_price

At this stage our expression is more complete (we know it is associated with lines), but its actually still contextless. Let's extend this by now asking for something more concrete, the SUM.

In [8]:
%%pydough

total_price = SUM(line_price)

Now this expression is more meaningful. If we assign this statement to global context, our actual TPCH graph, then we can compute the total price across all lines.

In [9]:
%%pydough

pydough.to_df(TPCH(total_line_price=total_price))

,total_line_price
0,2.181022e+11


In practice though, this may not solve our core question. Instead, we may want to apply a different **context**, say for example total_price for each order. We can instead represent that as follows.

In [10]:
%%pydough

order_total_price = orders(total_line_price=total_price)
pydough.to_df(order_total_price)

,total_line_price
0,167183.2296
1,44694.4600
2,190548.7248
3,29770.1730
4,137930.6046
...,...
1499995,111787.8756
1499996,68224.3200
1499997,88356.4944
1499998,62681.1456


Notice that are able to reuse the exact same code, but by swapping the context we can ultimately modify the semantics. This makes testing the underlying behavior much more scalable. To ask is this statement correct, we can instead compose our question to ask:
* Is this underlying expression correct?
* Is this context correct?

Since these can be verify independently, we can develop greater confidence in our question since it arises from composable building blocks. We could also generate selected contexts to build clear testing.

In [11]:
%%pydough

# Compute the sum of the first 5 line numbers, which can be known for testing.
top_five_lines = lines.TOP_K(5, by=line_number.ASC())
top_five_line_price = TPCH(total_price=SUM(top_five_lines(line_price=price_def).line_price))
pydough.to_df(top_five_line_price)

/Users/nicholasriasanovsky/Documents/bodo/PyDough/pydough/sqlglot/sqlglot_relational_visitor.py:215: UserWarning: PyDough when using SQLITE dialect does not support ascending ordering with nulls last (changed to nulls first)
  warnings.warn(


,total_price
0,168805.6798


Now let's return to extending our question. Building able to compute order sums is great, but we care about results per line. As a result, now we can even extend our orders to an additional context within lines. We will once again define more defintions. Our ratio definition will now ask us to propagate our previous `total_price` price that we computed and compare it to the result of `price_def`.

In [12]:
%%pydough

ratio = price_def / BACK(1).total_price

Now we will build our final question by moving from the context of the orders to the context of orders and lines together. Since Orders -> Lines is a Many to One relationship, this places us in the context of lines with some additional information.

For actually fully solving our prior question, we will compute the ratio and then select 5 smallest ratio value, breaking ties with a combination of the order number and line number.

In [13]:
%%pydough

line_ratios = order_total_price.lines(revenue_ratio=ratio, key=order_key, line_number=line_number)
lowest_ratios = line_ratios.TOP_K(5, by=(revenue_ratio.ASC(),key.DESC(), line_number.DESC()))

In [14]:
pydough.to_df(lowest_ratios)

/Users/nicholasriasanovsky/Documents/bodo/PyDough/pydough/sqlglot/sqlglot_relational_visitor.py:215: UserWarning: PyDough when using SQLITE dialect does not support ascending ordering with nulls last (changed to nulls first)
  warnings.warn(


,revenue_ratio,key,line_number
0,0.002341,363876,1
1,0.002359,4920774,1
2,0.002368,3274400,4
3,0.002399,2230976,2
4,0.002400,497094,4
